# Definindo Regex para identificar as informações relevantes

In [1]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re

## Exemplos de texto que podem ser fornecidos

In [3]:
texto_1 = '''
VALIDO EM TODO TERRITORIO NACIONAL
07.116.299-9 expevis 20/09/1976
NOME
SANDLER CAFFARO
FILIACAO
GABRIELA AUDI LUCARELLI
NATURALIDADE N DATA DE NASCIMENTO
BORBA-AM É Do 20/08/1999
Doc ongem C.NAS=116 LV=291 FL=Sd3
CURITIBA-PR
CPF 025.762.835-51 71695/9247

TONELOTITEYTAN ROSANTE
NH 94 ASSINATURA DO DIRETOR
'''

In [4]:
%matplotlib inline

import pytesseract
import cv2

import pandas as pd
from matplotlib import pyplot as plt

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [5]:
img_path = "../../BID Dataset/RG_Verso/00025928_in.jpg"

In [6]:
image = cv2.imread(img_path)

In [7]:
df = pytesseract.image_to_data(image, lang='por', output_type='data.frame')

## Criando e testando Regex/ identificação de padrões

In [8]:
def create_info_dict_from_row(row, name):
    return {
        'left': row['left'],
        'top': row['top'],
        'width': row['width'],
        'height': row['height'],
        'name': name
    }

### Identificação CPF

In [9]:
cpf_regex = r'\d{3}[\,\.]?\d{3}[\,\.]?\d{3}[\,\.\-\/]?\d{2}'

def find_cpf(identified_texts_df):
    for i, row in identified_texts_df.iterrows():
        if is_cpf(row['text']):
            return format_cpf(row['text']), create_info_dict_from_row(row, 'cpf')

def is_cpf(text):
    if not isinstance(text, str):
        return False
    return bool(re.search(cpf_regex, text))

def format_cpf(unformated_cpf):
    # Remove todas pontuações da regex, mantendo apenas os digitos
    cpf = re.sub(r'[-,./]', '', unformated_cpf)
    # Retorna o CPF com o seu padrão correto
    return f'{cpf[0:3]}.{cpf[3:6]}.{cpf[6:9]}-{cpf[9:11]}'
    

In [10]:
print(is_cpf('123.456.789-10'))

True


In [11]:
find_cpf(df)

('025.762.835-51',
 {'left': 115, 'top': 407, 'width': 133, 'height': 14, 'name': 'cpf'})

### Identificação RG

In [12]:
rg_regex = r'\d{2}[\,\.]?\d{3}[\,\.]?\d{3}[\,\.\-]?\d'

def find_rg(identified_texts_df):
    for i, row in identified_texts_df.iterrows():
        if is_rg(row['text']):
            return format_rg(row['text']), create_info_dict_from_row(row, 'cpf')

def is_rg(text):
    if not isinstance(text, str):
        return False
    return bool(re.search(rg_regex, text))

def format_rg(unformated_rg):
    # Remove todas pontuações da regex, mantendo apenas os digitos
    rg = re.sub(r'[-,.]', '', unformated_rg)
    # Retorna o RG com o seu padrão correto
    return f'{rg[0:2]}.{rg[2:5]}.{rg[5:8]}-{rg[8:9]}'

### Identificação data de nascimento

In [13]:
date_regex = r'\d{2}[\/]?\d{2}[\/]?\d{4}'

def search_for_birthdate(identified_texts_df):
    dates_count = 0
    for i, row in identified_texts_df.iterrows():
        if is_date(row['text']):
            dates_count += 1
        if dates_count == 2:
            return format_date(row['text']), create_info_dict_from_row(row, 'data de nascimento')

def is_date(text):
    if not isinstance(text, str):
        return False
    return bool(re.search(date_regex, text))

def format_date(unformated_date):
    # Remove todas pontuações da regex, mantendo apenas os digitos
    date = re.sub(r'[/]', '', unformated_date)
    # Retorna a data formatada com o seu padrão correto
    return {
        'formated_date': f'{date[0:1]}/{date[2:3]}/{date[4:]}',
        'day': int(date[0:1]),
        'month': int(date[2:3]),
        'year': int(date[4:])
    }

### Identificação nome

In [14]:
def search_for_name(identified_texts_df):
    pass 

In [21]:
# init camera
camera = cv2.VideoCapture(0)

In [22]:
ret, frame = camera.read()

In [25]:
# show camera frame
from time import sleep
from IPython.display import display, Image

display_handle=display(None, display_id=True)
for i in range(1000):
    ret, frame = camera.read()
    _, frame = cv2.imencode('.jpeg', frame)
    display_handle.update(Image(data=frame.tobytes()))

None

In [26]:
camera.release()
display_handle.update(None)